In [5]:
#--------------------------------------------------------------------------------
# Author  : Jacob Winters (jwinterscs@gmail.com)
# Created Date: 04/17/22
# Version ='2.0'
# -------------------------------------------------------------------------------
# Jupyter Plotly Dash application to allow for visualization of a PyMongo dataset
# -------------------------------------------------------------------------------
# Imports
# -------------------------------------------------------------------------------
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# Import the Animal Shelter CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# User authentication
username = "aacuser"
password = "teambds"
shelter = AnimalShelter(username, password)

# Convert PyMongo cursor into Pandas DataFrame
df = pd.DataFrame.from_records(shelter.read_all({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('AAC Database')

# Load and then encode the client logo image
# Image filename directory will need to be updated per individual installation
image_filename = '/Users/jake/Desktop/GSLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Declare the application interfaces
app.layout = html.Div(
    [
        html.Hr(),
        html.Div(id='hidden-div', style={'display':'none'}),
        html.Center(html.A(html.Img(src="data:image/png;base64,{}".format(encoded_image.decode())))),
        html.Center(html.B(html.H1('Grazioso Salvare Animal Shelter Dashboard'))),
        html.Center(html.H3('By Jacob Winters')),
        html.Hr(),
        html.Div(
            [
                dcc.RadioItems(
                    id="filter-type",
                    options=[
                        {"label": "Water Rescue", "value": "WR"},
                        {"label": "Mountain Rescue", "value": "MR"},
                        {"label": "Disaster or Individual Tracking", "value": "DR"},
                        {"label": "Reset", "value": "R"},
                    ],
                    value="R",
                    labelStyle={"display": "inline-block"},
                ),
            ]
        ),
        html.Div(
            [
                dash_table.DataTable(
                    id="datatable-id",
                    columns=[
                        {"name": i, "id": i, "deletable": False, "selectable": True}
                        for i in df.columns
                    ],
                    editable=False,
                    row_deletable=False,
                    filter_action="native",
                    sort_action="native",
                    sort_mode="multi",
                    row_selectable="single",
                    page_size = 15
                ),
            ]
        ),
        html.Br(),
        html.Hr(),
        html.Div(className='row',
            style={'display' : 'flex'},
                children=[
        html.Div(
            id='datatable-id-container',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ]
)])

#############################################
# Interaction Between Components / Controller
#############################################

# Radio Buttons Callback Function
# -------------------------------
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(value):
    # Reset values to their default state
    if value == "R":
        df = pd.DataFrame.from_records(shelter.read_all()).to_dict("records")
        return df
    # Water rescue animal criteria filter
    if value == "WR":
        df = pd.DataFrame(list(shelter.read_all({'$and':[{'$or':[{"breed":"Labrador Retriever Mix"},
        {"breed":"Chesapeake Bay Retriever"},{"breed":"Newfoundland"}]},
        {"sex_upon_outcome":"Intact Female"},{"age_upon_outcome_in_weeks":{'$lte':156, '$gte':26}}]})))
        return df.to_dict("records")
    # Mountain rescue animal criteria filter
    if value == "MR":
        df = pd.DataFrame(list(shelter.read_all({'$and':[{'$or':[{"breed":"German Shepherd"},{"breed":"Alaskan Malamute"},
        {"breed":"Old English Sheepdog"},{"breed":"Siberian Husky"},{"breed":"Rottweiler"}]},
        {"sex_upon_outcome":"Intact Male"},{"age_upon_outcome_in_weeks":{'$lte':156, '$gte':26}}]})))
        return df.to_dict("records")
    # Disaster/individual tracking animal criteria filter
    if value == "DR":
        df = pd.DataFrame(list(shelter.read_all({'$and':[{'$or':[{"breed":"Doberman Pinscher"},{"breed":"German Shepherd"},
        {"breed":"Golden Retriever"},{"breed":"Bloodhound"},{"breed":"Rottweiler"}]},
        {"sex_upon_outcome":"Intact Male"},{"age_upon_outcome_in_weeks":{'$lte':300, '$gte':20}}]})))
        return df.to_dict("records")

# Column Highlight Callback Function
# ----------------------------------
@app.callback(
    Output("datatable-id", "style_data_conditional"), [Input("datatable-id", "selected_columns")]
)
    # Change the background color of the user-selected column
def update_styles(selected_columns):
    return [{"if": {"column_id": i}, "background_color": "#D2F3FF"} for i in selected_columns]

# Pie Chart Callback Function
# ---------------------------
@app.callback(
    Output('datatable-id-container', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_virtual_selected_rows"),],)
def update_graphs(derived_viewport_data, derived_virtual_selected_rows):
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows=[]    
    dff = df if derived_viewport_data is None else pd.DataFrame(derived_viewport_data)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    # Create our pie chart based on the animal breed information
    return [
       dcc.Graph(            
           figure = px.pie(
               data_frame = dff, 
               values = values, 
               names = names, 
               color_discrete_sequence = px.colors.sequential.RdBu,  
           )
       )
    ]

# Map Callback Function
# ---------------------
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_viewport_selected_rows")])
def update_map(derived_viewport_data, derived_viewport_selected_rows):
    if derived_viewport_selected_rows is None:
        derived_viewport_selected_rows=[]
    dff = pd.DataFrame.from_dict(derived_viewport_data)
    singleRow = dff.iloc[derived_viewport_selected_rows]
    # Geolocation map style parameters, map centered at Austin, TX coordinates
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[singleRow.iloc[0,13],singleRow.iloc[0,14]], children=[
                dl.Tooltip(singleRow['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(singleRow['name'])
                ])
            ])
        ])
    ]

app